In [1]:
import numpy as np     #只需要下载numpy库即可
import random
import GridWorld_v2

In [4]:
gamma = 0.9   #折扣因子，越接近0越近视

rows = 5      #记得行数和列数这里要同步改
columns = 5
# gridworld = GridWorld_v2.GridWorld_v2(rows=rows, columns=columns, forbiddenAreaNums=8, targetNums=2, seed = 52,forbiddenAreaScore=-10)
# gridworld = GridWorld_v2.GridWorld_v2(desc = [".#",".T"])             #赵老师4-1的例子
# gridworld = GridWorld_v2.GridWorld_v2(desc = ["##.T","...#","...."])  #随便弄的例子
gridworld = GridWorld_v2.GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = [".....",".##..","..#..",".#T#.",".#..."]) 
gridworld.show()

trajectorySteps = 100

value = np.zeros(rows*columns)       #初始化可以任意，也可以全0
qtable = np.zeros((rows*columns,5))  #初始化，这里主要是初始化维数，里面的内容会被覆盖所以无所谓

# np.random.seed(50)
policy = np.eye(5)[np.random.randint(0,5,size=(rows*columns))] 
print(gridworld.showPolicy)
gridworld.showPolicy(policy)
# gridworld.showPolicy(policy)

⬜️⬜️⬜️⬜️⬜️
⬜️🚫🚫⬜️⬜️
⬜️⬜️🚫⬜️⬜️
⬜️🚫✅🚫⬜️
⬜️🚫⬜️⬜️⬜️
<bound method GridWorld_v2.showPolicy of <GridWorld_v2.GridWorld_v2 object at 0x7f73e19b7610>>
🔄🔄⬆️⬇️➡️
⬆️⏫️⏫️🔄⬇️
➡️⬇️⏪⬅️➡️
🔄🔄✅🔄🔄
⬅️🔄🔄➡️➡️


In [5]:
#通过采样的方法计算action value，model free的话意味着不知道整个gridworld的概率了，所以不能直接套贝尔曼方程迭代求解
gridworld.showPolicy(policy)
print("random policy")
qtable = np.zeros((rows*columns,5))
qtable_pre = qtable.copy()+1
while(np.sum((qtable_pre-qtable)**2)>0.001):
    print(np.sum((qtable_pre-qtable)**2))
    qtable_pre = qtable.copy()
    #通过采样获得action-value的值
    for i in range(rows*columns):
        for j in range(5):
            Trajectory = gridworld.getTrajectoryScore(nowState=i, action=j, policy=policy, steps=trajectorySteps)
            
            # 注意这里的返回值是大小为(trajectorySteps+1)的元组列表，因为把第一个动作也加入进去了
            # a = r + gamma*r1 + gamma*gamma*r2 + gamma*gamma*gamma*r3 ……
            # 返回值是 S A R
            
            tmp = Trajectory[trajectorySteps][2]
            for k in range(trajectorySteps-1,-1,-1):
                tmp = tmp*gamma + Trajectory[k][2]  #细节从后往前优化算法
            # print(tmp)
            
            qtable[i][j] = tmp #这里是通过采样，获得action value的值

    policy = np.eye(5)[np.argmax(qtable,axis=1)]
    print(qtable[17])
    print(qtable[22])
    # policy = np.eye(5)[np.argmax(qtable,axis=1)]  #qtable的最优值作为更新策略，并用独热码来表示
    gridworld.showPolicy(policy)
    print(np.sum((qtable_pre-qtable)**2))


🔄🔄⬆️⬇️➡️
⬆️⏫️⏫️🔄⬇️
➡️⬇️⏪⬅️➡️
🔄🔄✅🔄🔄
⬅️🔄🔄➡️➡️
random policy
125.0
[-90.99760947 -99.99760947   0.         -99.99760947 -88.99760947]
[-88.99760947  -8.09976095  -1.         -99.99760947   0.        ]
➡️⬅️➡️⬇️⬅️
⬆️⏫️⏩️⬆️⬅️
⬆️⬆️⏫️⬆️⬇️
🔄⏪✅⏩️🔄
⬆️⏩️🔄⬅️⬆️
341292.92396520893
341292.92396520893
[-19. -10.   0. -10.   1.]
[  1.   0.  -1. -10.   0.]
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬆️
⬆️⬅️⏬⬆️⬆️
⬆️⏩️✅⏪⬆️
⬆️⏩️⬆️➡️⬆️
290060.87700286834
290060.87700286834
[-1.00023905 -1.00023905  8.99976095 -1.00023905  9.99976095]
[ 9.99976095  0.          7.99976095 -1.90023905  8.99976095]
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬆️
⬆️⬅️⏬⬆️⬆️
⬆️⏩️✅⏪⬆️
⬆️⏩️⬆️⬅️⬆️
3972.9015500560045
3972.9015500560045
[-1.00023905 -1.00023905  8.99976095 -1.00023905  9.99976095]
[ 9.99976095  8.09976095  7.99976095 -1.90023905  8.99976095]
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬆️
⬆️⬅️⏬⬆️⬆️
⬆️⏩️✅⏪⬆️
⬆️⏩️⬆️⬅️⬅️
328.0306370259419
328.0306370259419
[-1.00023905 -1.00023905  8.99976095 -1.00023905  9.99976095]
[ 9.99976095  8.09976095  7.99976095 -1.90023905  8.99976095]
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬆